In [90]:
""" Implementation of chatbot that has access to 
our Pinecone database of lyrics. """
# Imports
import os
import pinecone
import openai
from dotenv import load_dotenv

# Get API keys
load_dotenv()

# Connect to Pinecone
pinecone.init(api_key=os.getenv("PINECONE_KEY2"), environment=os.getenv("PINECONE_ENV2"))
index = pinecone.Index(index_name="lyrics")

# Connect to OpenAI
openai.api_key = os.getenv("OPENAI_KEY2")
openai.organization = os.getenv("OPENAI_ORG2")

In [28]:
# Load the lyrics from the .json file
import json
with open('./lyrics.json') as f:
    lyrics = json.loads(f.read())

In [18]:
from typing import Optional, List
from pydantic import BaseModel, Field
# Create a class for the lyrics object
class SongClip(BaseModel):
    """ SongClip object. """
    lyrics: Optional[str] = Field(description="Lyrics of the song")
    title: str = Field(description="Title of the song")
    artist: str = Field(description="Artist of the song")
    album: Optional[str] = Field(description="Album of the song")
    source: str = Field(description="Source of the song")
    music: Optional[List[float]] = Field(
                description="Music of the song")
    description: Optional[str] = Field(
                description="Description of the song")

In [48]:
from langchain.embeddings import OpenAIEmbeddings
# Create a function to embed the lyrics and upsert
# them into our Pinecone index
def upsert_lyrics(songs_list: List[SongClip]):
    """ Upsert the lyrics into the Pinecone index. """
    embeddings = OpenAIEmbeddings()
        
    # Iterate through the songs and embed the lyrics
    for song in songs_list:
        # Embed the lyrics
        lyrics_emb = embeddings.embed_documents(song.lyrics)
        
        # Create the metadata
        metadata = {"title": song.title,
                    "artist": song.artist,
                    "album": song.album,
                    "source": song.source,
                    "description": song.description,
                    "lyrics": song.lyrics,
                }
        values = lyrics_emb[0]
        song_id = song.title

        # Define the vectors
        vectors = [
            {
            "metadata": metadata,
            "values": values,
            "id": song_id,
            }
        ]

        # Upsert the lyrics
        index.upsert(vectors=vectors)

    # Return a success message and the index name statistics
    return f"Successfully upserted {len(songs_list)} songs into the index."
        
# Save the lyrics_dict_list to a json file
def save_lyrics(songs_list: List[SongClip]):
    """ Save the lyrics to a json file. """
    # Create a dictionary to store the lyrics
    lyrics_dict = {}
    # Iterate through the songs and add them to the dictionary
    for song in songs_list:
        lyrics_dict[song.title] = song.dict()
    # Save the lyrics to a json file
    with open("lyrics.json", "w") as f:
        json.dump(lyrics_dict, f)
    # Return a success message
    return f"Successfully saved {len(songs_list)} songs to lyrics.json"

In [41]:
songs_list = [song for song in lyrics.values()]
    

In [44]:
# Convert the songs_list values to SongClip objects
songs_list = [SongClip(**song) for song in songs_list]

In [51]:
# For each song in the songs_list, embed the lyrics and create the metadata
# for the Pinecone index
upsert_lyrics(songs_list)



ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json', 'date': 'Sat, 14 Oct 2023 18:40:02 GMT', 'x-envoy-upstream-service-time': '0', 'content-length': '60', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"not supported value type","details":[]}


In [52]:
from langchain.embeddings import OpenAIEmbeddings

# Define a function to perform search on Pinecone
def search(query: str, index=index, embeddings=OpenAIEmbeddings()):
    """ Perform search on Pinecone. """
    # Embed the query
    query = embeddings.embed_query(query)
    # Perform search
    results = index.query(queries=[query], top_k=3, include_metadata=True)
    # Get song
    return results

search_results = search("I want to write a song about love")
    

In [85]:
def convert_results_to_dicts(search_results):
    dicts_list = []
    results_dict = search_results.to_dict()
    for result in results_dict["results"][0]["matches"]:
        song = SongClip(**result["metadata"])
        dicts_list.append(song.dict())
        return dicts_list   
    

In [86]:
results_dict_list = convert_results_to_dicts(search_results)

In [87]:
print(results_dict_list)

[{'lyrics': '11 ContributorsGrey Street Lyrics[Verse 1]\nOh, look at how she listens\nShe says nothing of what she thinks\nShe just goes stumbling through her memories\nStaring out onto Grey Street\nShe thinks, "Hey, how did I come to this?\nI dream myself a thousand times around the world\nBut I can\'t get out of this place"\n\n[Chorus]\nThere\'s an emptiness inside her\nAnd she\'d do anything to fill it in\nBut all the colors mix together - to grey\nAnd it breaks her heart\n\n[Verse 2]\nHow she wishes it was different\nShe prays to God most every night\nAnd though she swears it doesn\'t listen\nThere\'s still a hope in her it might\nShe says, "I pray\nBut they fall on deaf ears\nAm I supposed to take it on myself?\nTo get out of this place?"\n\n[Chorus]\nThere\'s loneliness inside her\nAnd she\'d do anything to fill it in\nAnd though it\'s red blood bleeding from her now\nIt feels like cold blue ice in her heart\nWhen all the colors mix together - to grey\nAnd it breaks her heart\nSe

In [102]:
# Create an openai completion function
def lyrics_chat(query: str):
    """ Function to facilitate the chat between the user and the chatbot. """
    # Perform search
    search_results = search(query)
    # Convert the results to a dictionary
    results_dict_list = convert_results_to_dicts(search_results)
    # Construct the prompt
    messages = [
        {"role": "system", "content": f"""You are a musician in the style of Dave Matthews that is engaged
        in a co-writing session with a fellow artist or fan. The user's most recent query is {query}.
        The most relevant of your songs to their query in list form is {results_dict_list}.  Please write a
        thoughtful response that is in the style of Dave Matthews and makes the user feel that you are
        truly engaged in the co-writing session with them. Keep the conversation open-ended and engaging, and
        think about how your songs in the song list may enrich the conversation.)"""},
        {"role": "user", "content": query},
        {"role": "system", "content": "The musician responds:"},
    ]
    # Create a list of models to iterate through
    models = ["gpt-4-0613", "gpt-4", "gpt-3.5-turbo-16k-0613", "gpt-3.5-turbo-16k, gpt-3.5-turbo-0613", "gpt-3.5-turbo"]
    for model in models:
        try:
            response = openai.ChatCompletion.create(
                messages=messages,
                model=model,
                max_tokens=250,
                temperature=0.7,
                top_p=1,
                frequency_penalty=0.5
            )
            artist_response = response.choices[0].message.content

            # Return the artist response
            return artist_response
        except TimeoutError as e:
            print(e)
            continue

In [103]:
answer = lyrics_chat("I want to write a song about love")

In [104]:
answer

"That's a wonderful idea! Love is such a universal theme, yet it's so unique to every individual's experience. It can be as colorful and vibrant as a rainbow, but at times, just as the song 'Grey Street' suggests, it can mix together into shades of grey.\n\nJust like in 'Grey Street', there's an exploration of yearning and the complexity of emotions. Perhaps in our love song, we could dive into these depths too.\n\nWhat kind of love are you thinking about? Is it the passionate, all-consuming kind? Or maybe it's a gentle, steady love that grows over time? Or perhaps it's about lost love and the pain that comes with it?\n\nRemember, there are no wrong answers here. All emotions are valid and beautiful in their own way. Let's dive in deeper and see where this journey takes us."